<a href="https://colab.research.google.com/github/CarbonicKevin/APS360-2020Summer-Project/blob/master/APS360_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Model v1

#Pull from Github

Find git token: user profile > settings > developer settings > Personal Access tokens. [Scope:](https://developer.github.com/apps/building-oauth-apps/understanding-scopes-for-oauth-apps/) repo.

In [17]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/...'       # path to your project on Drive

GIT_USERNAME   = ""         #@param {type: "string"}
GIT_TOKEN      = ""         #@param {type: "string"}
GIT_REPOSITORY = "APS360-2020Summer-Project"


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"I    # in case we haven't created it already   

GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/...I’: File exists
Cloning into '{GIT_REPOSITORY}'...
fatal: unable to access 'https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git/': The requested URL returned error: 400
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
rsync: change_dir "/content/drive/My Drive/..." failed: No such file or directory (2)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1196) [sender=3.1.2]


In [ ]:
# import library into google colab
import imp 
lib = imp.new_module('code/train_AI_lib.py')
exec(open("./code/train_AI_lib.py").read(), lib.__dict__)

## Model

In [ ]:
dataPath = ''
imgPath  = ''
img_dataset = imgLoader(dataPath=dataPath, imgPath=imgPath)